In [3]:
import io
import multiprocessing
import pandas as pd

In [4]:
df = pd.read_csv('data.csv',index_col=0)

In [5]:
df.head()

,caption,img
0,captions_resized_1M/cities_instagram/melbourne...,img_resized_1M/cities_instagram/melbourne/1486...
1,captions_resized_1M/cities_instagram/melbourne...,img_resized_1M/cities_instagram/melbourne/1487...
2,captions_resized_1M/cities_instagram/melbourne...,img_resized_1M/cities_instagram/melbourne/1482...
3,captions_resized_1M/cities_instagram/melbourne...,img_resized_1M/cities_instagram/melbourne/1490...
4,captions_resized_1M/cities_instagram/melbourne...,img_resized_1M/cities_instagram/melbourne/1490...


In [6]:
captions = list(df['caption'])

In [7]:
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string,strip_non_alphanum,strip_numeric

In [8]:
class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        for file in captions:
            for line in open(f'{file}'):
                CUSTOM_FILTERS = [lambda x: x.lower(), remove_stopwords, strip_non_alphanum,strip_numeric]
                yield preprocess_string(line,CUSTOM_FILTERS)

In [9]:
import gensim.models

In [12]:
cores=multiprocessing.cpu_count() -1
cores

63

In [23]:
sentences = MyCorpus()
model = gensim.models.Word2Vec(workers=cores,
                               window = 2,
                               vector_size=300,
                               min_count=6,
                               sample=1e-3)

INFO - 14:45:25: Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.025)', 'datetime': '2021-06-05T14:45:25.249069', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 07:30:14) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-72-generic-x86_64-with-glibc2.10', 'event': 'created'}


In [24]:
from time import time

In [25]:
t = time()

model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 14:45:29: collecting all words and their counts
INFO - 14:45:29: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 14:45:29: PROGRESS: at sentence #10000, processed 206729 words, keeping 48905 word types
INFO - 14:45:30: PROGRESS: at sentence #20000, processed 412245 words, keeping 77469 word types
INFO - 14:45:30: PROGRESS: at sentence #30000, processed 616580 words, keeping 100726 word types
INFO - 14:45:31: PROGRESS: at sentence #40000, processed 826143 words, keeping 121156 word types
INFO - 14:45:32: PROGRESS: at sentence #50000, processed 1030141 words, keeping 138811 word types
INFO - 14:45:32: PROGRESS: at sentence #60000, processed 1238064 words, keeping 155332 word types
INFO - 14:45:33: PROGRESS: at sentence #70000, processed 1447986 words, keeping 170574 word types
INFO - 14:45:33: PROGRESS: at sentence #80000, processed 1655636 words, keeping 185268 word types
INFO - 14:45:34: PROGRESS: at sentence #90000, processed 1859949 words, keeping 1992

Time to build vocab: 1.15 mins


In [22]:
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [26]:
t = time()

model.train(sentences, total_examples=model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 14:47:23: Word2Vec lifecycle event {'msg': 'training model with 63 workers on 211464 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2', 'datetime': '2021-06-05T14:47:23.903856', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 07:30:14) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-72-generic-x86_64-with-glibc2.10', 'event': 'train'}
INFO - 14:47:24: EPOCH 1 - PROGRESS: at 0.86% examples, 159900 words/s, in_qsize 0, out_qsize 0
INFO - 14:47:25: EPOCH 1 - PROGRESS: at 1.89% examples, 171872 words/s, in_qsize 0, out_qsize 0
INFO - 14:47:27: EPOCH 1 - PROGRESS: at 2.57% examples, 155150 words/s, in_qsize 0, out_qsize 0
INFO - 14:47:28: EPOCH 1 - PROGRESS: at 3.63% examples, 163991 words/s, in_qsize 0, out_qsize 0
INFO - 14:47:29: EPOCH 1 - PROGRESS: at 4.65% examples, 166995 words/s, in_qsize 0, out_qsize 0
INFO - 14:47:30: EPOCH 1 - PROGRESS: at 5.66% examples, 169605 words/s, in_qsize 0, out_qsize 0
INFO - 14:47:31: EPOCH 1 - PROGRESS: at 6.62% e

Time to train the model: 57.32 mins


In [27]:
model.save('word2vec.fullmodel')

INFO - 15:45:55: Word2Vec lifecycle event {'fname_or_handle': 'word2vec.fullmodel', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-06-05T15:45:55.605160', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 07:30:14) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-72-generic-x86_64-with-glibc2.10', 'event': 'saving'}
INFO - 15:45:55: storing np array 'vectors' to word2vec.fullmodel.wv.vectors.npy
INFO - 15:45:55: storing np array 'syn1neg' to word2vec.fullmodel.syn1neg.npy
INFO - 15:45:55: not storing attribute cum_table
INFO - 15:45:55: saved word2vec.fullmodel
